In [464]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


keras.utils.set_random_seed(253)

In [465]:
def yajuns_function(t: np.float32):
    return(-1 * np.power(np.e, t)) + np.power(np.e, 2*t) 

t_test = np.arange(0, 5, .01, dtype='float32')
y_test = np.array([yajuns_function(t) for t in t_test], dtype='float32')


In [466]:
# residual is how accurate the output is with respect to target equation
def calc_residual(model: keras.Model, alpha: tf.Variable, beta: tf.Variable, t_vals)-> tf.Tensor:
    
    t_vals_tensor = tf.convert_to_tensor(t_vals, dtype='float32')

    with tf.GradientTape(persistent= True) as gt:

        gt.watch(t_vals_tensor)

        y = model(t_vals_tensor)
        
        # predicted values
        y_prime = gt.gradient(y, t_vals_tensor)
        
        y = tf.convert_to_tensor(tf.squeeze(y), dtype='float32')

    y_dprime = gt.gradient(y_prime, t_vals_tensor) # second derivative of predicted values
    del gt

    # print("\n")
    # print("t: "+ str(type(t_vals_tensor)))
    # print("y: " + str(y))
    # print("dt: " + str(tf.reduce_mean(y_prime)))
    # print("d2t: " + str(y_dprime))
    # print("residual: " + str(tf.reduce_mean(tf.add(y_dprime, tf.add(tf.scalar_mul(alpha, y_prime), tf.scalar_mul(beta, y))))))
    # print("\n")
    
    return tf.add(y_dprime, tf.add(tf.scalar_mul(alpha, y_prime), tf.scalar_mul(beta, y)))


In [467]:
# calculate how accurate the model is with respect to the actual answers
def calc_loss(model: keras.Model, alpha: tf.Variable, beta: tf.Variable, t_vals):
    """
    Custom loss function made for neural network. It calculates the residual, and then
    adds it to the difference in the boundary outputs and predicted values.
    Returns the loss.
    
    """

    # need residual for loss, it will tell us how accurate vals are with respect to itself
    residual = calc_residual(model, alpha, beta, t_vals)  
    av_residual = tf.reduce_mean(tf.square(residual))

    t = tf.convert_to_tensor(np.array([0.0], dtype = 'float32'))
    with tf.GradientTape() as gt:
        gt.watch(t)
        y = model(t) # y(0) should equal 0
        
    dy_dt = gt.gradient(y, t) # y'(0) should equal 1    

    
    # print("\n")
    # print("average residual: " + str(av_residual))
    # print("y: " + str(y))
    # print("dy_dt: " + str(dy_dt))
    # print("dy_dt -1: " + str(dy_dt - 1))
    # print("returns: " + str(tf.add(av_residual, tf.add(tf.pow(y, 2), tf.pow(tf.subtract(dy_dt, 1), 2)))))
    # print("\n")
    
    

    return tf.add(av_residual, tf.add(tf.square(y), tf.square(tf.subtract(dy_dt, 1)))) # residual should equal 0, y should equal 0, and dt should equal 1 

In [468]:
# gradient is of the loss function with respect to all the variables in the model
def calc_gradient(model: keras.Model, alpha: tf.Variable, beta: tf.Variable, t_vals):
    """
    Calculates the gradient to apply to the neural network. Returns the
    gradient.
    
    """

    with tf.GradientTape() as gt:
        gt.watch(model.trainable_variables) # weights
        loss = calc_loss(model, alpha, beta, t_vals)

    gradient = gt.gradient(loss, model.trainable_variables)

    # print(str(gradient))

    return loss, gradient

In [469]:
def train(model: keras.Model, alpha: tf.Variable, beta: tf.Variable, t_vals, optimizer: keras.optimizers.Optimizer):
    """
    Executes one training step for the network. Calculates loss and gradient, applies the 
    gradient, then returns the loss.
    
    """
    
    loss, gradient = calc_gradient(model, alpha, beta, t_vals)

    optimizer.apply_gradients(zip(gradient, model.trainable_variables))

    return loss

In [470]:
# make layers of the model
# most accurate 2 period wave is
# 20, 15, 10, 5 with .5 dropout in between

def make_network() -> tuple:
    layers = []         

    layers.append(keras.layers.Dense(16, activation = 'elu', input_shape = (1,)))
    layers.append(keras.layers.Dense(16, activation = 'elu'))
    layers.append(keras.layers.Dense(16, activation = 'elu'))

    output_layer = keras.layers.Dense(1, activation = 'linear', name = "output")
    layers.append(output_layer)

    output_layer.add_weight(shape = (), dtype='float32', trainable=True, name = 'alpha')
    output_layer.add_weight(shape = (), dtype='float32', trainable=True, name = 'beta')
    
    network = keras.Sequential(layers)

    #print(output_layer.trainable_weights)

    return (network, output_layer)


In [471]:
# set up batches
batch_size = 1000000

t_batches = []

i = 0
while i < (len(t_test) / batch_size):
    t_batches.append(np.array(t_test[i * batch_size : (i * batch_size) + batch_size], dtype='float32'))
    i += 1


In [472]:
# train the model
epochs = 1000
i = 0
network, output_layer = make_network()
alpha = output_layer.trainable_weights[0]
beta = output_layer.trainable_weights[1]

while i < epochs:

    loss = 0
    accuracy = 0
    j = 0

    while j < len(t_batches): # train each batch per epoch
        loss += train(network, alpha, beta, t_batches[j], keras.optimizers.Adam())
        j += 1


    print("Epoch: " + str(i))
    print("Loss: " + str(loss / j))
    print("Alpha: " + str(alpha))
    print("Beta: " + str(beta))
    print("\n")
    # save every epoch so i can jump ship at any time and still have a network to show for it
    network.save_weights("alpha_beta_problem.weights.h5")

    i += 1

Epoch: 0
Loss: tf.Tensor([[1.1323166]], shape=(1, 1), dtype=float32)
Alpha: <tf.Variable 'alpha:0' shape=() dtype=float32, numpy=-0.46383363>
Beta: <tf.Variable 'beta:0' shape=() dtype=float32, numpy=1.1994174>


Epoch: 1
Loss: tf.Tensor([[0.9472865]], shape=(1, 1), dtype=float32)
Alpha: <tf.Variable 'alpha:0' shape=() dtype=float32, numpy=-0.46483362>
Beta: <tf.Variable 'beta:0' shape=() dtype=float32, numpy=1.1984173>


Epoch: 2
Loss: tf.Tensor([[0.7336477]], shape=(1, 1), dtype=float32)
Alpha: <tf.Variable 'alpha:0' shape=() dtype=float32, numpy=-0.46583357>
Beta: <tf.Variable 'beta:0' shape=() dtype=float32, numpy=1.1974174>


Epoch: 3
Loss: tf.Tensor([[0.58065057]], shape=(1, 1), dtype=float32)
Alpha: <tf.Variable 'alpha:0' shape=() dtype=float32, numpy=-0.4668335>
Beta: <tf.Variable 'beta:0' shape=() dtype=float32, numpy=1.1964175>


Epoch: 4
Loss: tf.Tensor([[0.4684463]], shape=(1, 1), dtype=float32)
Alpha: <tf.Variable 'alpha:0' shape=() dtype=float32, numpy=-0.46783242>
Beta: 

KeyboardInterrupt: 

In [ ]:
network, output_layer = make_network()
network.load_weights("alpha_beta_problem.weights.h5", skip_mismatch=True)
x = np.arange(0, 5, .01, dtype='float32')
prediction = network.predict(x, verbose = 0)

y = np.squeeze(prediction)
# y = list(map(lambda a: a*10,y)) # magnification :(

plt.plot(t_test, y_test) # blue line, actual sine wave
plt.plot(x, y, "r--") # red dashes, neural net prediction
plt.axis((0, 7, -1, 10000))
plt.show()

print(y_test)
print(y)